In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import root_mean_squared_error

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experimental")

2025/05/24 00:20:38 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experimental' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/mlruns/2', creation_time=1748046038801, experiment_id='2', last_update_time=1748046038801, lifecycle_stage='active', name='nyc-taxi-experimental', tags={}>

In [3]:
!mlflow --version

mlflow, version 2.22.0


In [4]:
df = pd.read_parquet("../data/green/green_tripdata_2023-01.parquet")

In [5]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.dt.total_seconds() / 60
df = df[df.duration >= 1 & (df.duration < 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [10]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "pastor")
    mlflow.log_param("train-data-path", "../data/green/green_tripdata_2023-01.parquet")
    
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer()
    
    X_train = dv.fit_transform(train_dicts)
    
    target = 'duration'
    y_train = df[target].values
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_train)
    
    rmse = root_mean_squared_error(y_train, y_pred)
    mlflow.log_metric("rmse", rmse)